In [1]:
import os
os.chdir('/home/gauravk/Documents/DataScience/Datasets/Ongoing Competitions/predict the roadsign/')

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [84]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [7]:
train.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target)
0,2c9180975a056a64015a1e0a52e57021,Rear,195,1.02,46,45,Rear
1,2c9180975a056a64015a1e17b32171e4,Rear,203,1.09,59,54,Rear
2,2c9180975a056a64015a1de4deb16bd5,Front,26,0.96,104,108,Front
3,2c9180975a056a64015a1de4deb16bdd,Rear,199,0.81,38,47,Rear
4,2c9180975a056a64015a1de4deb16bd6,Rear,208,0.93,54,58,Rear


In [6]:
test.shape

(31485, 6)

In [85]:
train[train['DetectedCamera'] != train['SignFacing (Target)']]

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target)
6,2c9180975a056a64015a1e01fd466f56,Left,245,0.93,109,117,Rear
9,2c9180975a056a64015a1e01fd466f51,Right,74,0.51,53,103,Front
14,2c9180975a056a64015a1e04ad9f6f8b,Left,258,0.48,39,81,Rear
21,2c9180975a056a64015a1e04ad9f6f83,Right,49,1.04,127,122,Front
27,2c9180975a056a64015a1e1d735c7293,Right,76,0.64,65,102,Front
29,2c9180975a056a64015a1e41bbc2775f,Left,253,0.69,37,54,Rear
30,2c9180975a056a64015a1e41bbc27758,Left,248,0.75,44,59,Rear
31,2c9180975a056a64015a1e41bbc27756,Right,72,0.55,52,94,Front
33,2c9180975a056a64015a1e13b6c7713e,Left,253,0.43,37,87,Rear
37,2c9180975a056a64015a1dceefde69de,Right,43,1.20,200,167,Front


In [16]:
train['SignFacing (Target)'].value_counts()

Front    21327
Rear     15662
Right      821
Left       675
Name: SignFacing (Target), dtype: int64

In [86]:
#encode as integer
mapping = {'Front':0, 'Right':1, 'Left':2, 'Rear':3}
train = train.replace({'DetectedCamera':mapping})
test = test.replace({'DetectedCamera':mapping})

In [87]:
#renaming column
train.rename(columns = {'SignFacing (Target)': 'Target'}, inplace=True)

In [88]:
#encode Target Variable based on sample submission file
mapping = {'Front':0, 'Left':1, 'Rear':2, 'Right':3}
train = train.replace({'Target':mapping})

In [89]:
#target variable
y_train = train['Target']
test_id = test['Id']

In [90]:
#drop columns
train.drop(['Target','Id'], inplace=True, axis=1)
test.drop('Id',inplace=True,axis=1)

In [23]:
#train model
clf = RandomForestClassifier(n_estimators=500,max_features=3,min_samples_split=5,oob_score=True, verbose=10)
clf.fit(train, y_train)

building tree 1 of 500
building tree 2 of 500


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s


building tree 3 of 500
building tree 4 of 500


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.5s remaining:    0.0s


building tree 5 of 500
building tree 6 of 500


...

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    1.2s remaining:    0.0s


building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
building tree 43 of 500
building tree 44 of 500
building tree 45 of 500
building tree 46 of 500
building tree 47 of 500
building tree 48 of 500
building tree 49 of 500
building tree 50 of 500
building tree 51 of 500
building tree 52

building tree 343 of 500
building tree 344 of 500
building tree 345 of 500
building tree 346 of 500
building tree 347 of 500
building tree 348 of 500
building tree 349 of 500
building tree 350 of 500
building tree 351 of 500
building tree 352 of 500
building tree 353 of 500
building tree 354 of 500
building tree 355 of 500
building tree 356 of 500
building tree 357 of 500
building tree 358 of 500
building tree 359 of 500
building tree 360 of 500
building tree 361 of 500
building tree 362 of 500
building tree 363 of 500
building tree 364 of 500
building tree 365 of 500
building tree 366 of 500
building tree 367 of 500
building tree 368 of 500
building tree 369 of 500
building tree 370 of 500
building tree 371 of 500
building tree 372 of 500
building tree 373 of 500
building tree 374 of 500
building tree 375 of 500
building tree 376 of 500
building tree 377 of 500
building tree 378 of 500
building tree 379 of 500
building tree 380 of 500
building tree 381 of 500
building tree 382 of 500


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.1min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=3, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=5, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=True, random_state=None,
            verbose=10, warm_start=False)

In [24]:
#predict on test data
pred = clf.predict_proba(test)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    6.3s finished


In [96]:
newDF = pd.DataFrame()
newDF = train

In [99]:
mini = newDF.SignWidth.min()
maxi = newDF.SignWidth.max()
mean = newDF.SignWidth.mean()
newDF.SignWidth = (newDF.SignWidth- mean)/(maxi- mini)
test.SignWidth = (test.SignWidth- mean)/(maxi- mini)

In [103]:
mini = newDF.AngleOfSign.min()
maxi = newDF.AngleOfSign.max()
mean = newDF.AngleOfSign.mean()
newDF.AngleOfSign = (newDF.AngleOfSign- mean)/(maxi- mini)
test.AngleOfSign = (test.AngleOfSign- mean)/(maxi- mini)

In [104]:
mini = newDF.SignHeight.min()
maxi = newDF.SignHeight.max()
mean = newDF.SignHeight.mean()
newDF.SignHeight = (newDF.SignHeight- mean)/(maxi- mini)
test.SignHeight = (test.SignHeight - mean)/(maxi- mini)

In [100]:
newDF = pd.get_dummies(newDF, columns=['DetectedCamera'])

In [94]:
test = pd.get_dummies(test, columns=['DetectedCamera'])

In [113]:
train

,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,3,195,1.02,-0.082321,45
1,3,203,1.09,-0.059514,54
2,0,26,0.96,0.019433,108
3,3,199,0.81,-0.096356,47
4,3,208,0.93,-0.068286,58
5,0,29,0.83,-0.048988,78
6,2,245,0.93,0.028205,117
7,3,204,1.04,-0.064777,54
8,0,25,0.95,0.019433,109
9,1,74,0.51,-0.070040,103


In [83]:
newDF.isnull().sum()

AngleOfSign         0
SignAspectRatio     0
SignWidth           0
SignHeight          0
DetectedCamera_0    0
DetectedCamera_1    0
DetectedCamera_2    0
DetectedCamera_3    0
dtype: int64

In [116]:
#write submission file and submit
columns = ['Front','Left','Rear','Right']
sub = pd.DataFrame(data=pred, columns=columns)
sub['Id'] = test_id
sub = sub[['Id','Front','Left','Rear','Right']]
sub.to_csv("sub_nn_5.csv", index=False) #99.8XXX

In [106]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

In [107]:

SS = StratifiedShuffleSplit(n_splits=3, test_size=0.4, random_state=0)

In [114]:
b = 100
for i, j in SS.split(newDF, y_train):# 
    nnClf = MLPClassifier(hidden_layer_sizes=(10,15,10), activation='relu', max_iter=250,  tol=1e-5, verbose=10)
    nnClf.fit(newDF.loc[i], y_train.loc[i])
    a = accuracy_score(nnClf.predict(newDF.loc[j]), y_train.loc[j])
    print"accur", a 

Iteration 1, loss = 0.91735220
Iteration 2, loss = 0.34079833
Iteration 3, loss = 0.28460354
Iteration 4, loss = 0.26547585
Iteration 5, loss = 0.25081728
Iteration 6, loss = 0.23543585
Iteration 7, loss = 0.22165298
Iteration 8, loss = 0.21019077
Iteration 9, loss = 0.20036457
Iteration 10, loss = 0.19127207
Iteration 11, loss = 0.18372993
Iteration 12, loss = 0.17772775
Iteration 13, loss = 0.17162143
Iteration 14, loss = 0.16615233
Iteration 15, loss = 0.16229987
Iteration 16, loss = 0.15871638
Iteration 17, loss = 0.15540046
Iteration 18, loss = 0.15224010
Iteration 19, loss = 0.14886853
Iteration 20, loss = 0.14585471
Iteration 21, loss = 0.14284352
Iteration 22, loss = 0.14040185
Iteration 23, loss = 0.13832060
Iteration 24, loss = 0.13601434
Iteration 25, loss = 0.13395756
Iteration 26, loss = 0.13267184
Iteration 27, loss = 0.13220407
Iteration 28, loss = 0.13050997
Iteration 29, loss = 0.12997482
Iteration 30, loss = 0.12836663
Iteration 31, loss = 0.12776198
Iteration 32, los

In [115]:
pred = nnClf.predict_proba(test)